## Training Script

This notebook serves as a reference for SFT for following 9 indian languages + english language.

1. Hindi
2. Telugu
3. Tamil
4. Kannada
5. Malayalam
6. Gujarati
7. Odia
8. Bengali
9. Punjabi
10. English

We use [unsloth](https://github.com/unslothai/unsloth) library for fine-tuning the `gemma-7b` model.

### Installation

In [ ]:
!pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git" wandb

### wandb login

In [ ]:
!wandb login '<wandb api key>'

### HuggingFace login

In [ ]:
from huggingface_hub import notebook_login

# This will prompt you to enter your token
notebook_login()

### Download Model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use for 4bit quantization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-7b", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = '<HF_TOKEN>',
)

### LoRA setup

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Test tokens for each language

In [1]:
# English reference text

text = """Google News is a news aggregator service developed by Google. It presents a continuous flow of links to articles organized from thousands of publishers and magazines. Google News is available as an app on Android, iOS, and the Web. Google released a beta version in September 2002 and the official app in January 2006."""

print(text)

Google News is a news aggregator service developed by Google. It presents a continuous flow of links to articles organized from thousands of publishers and magazines. Google News is available as an app on Android, iOS, and the Web. Google released a beta version in September 2002 and the official app in January 2006.


In [ ]:
hindi_text = """Google News Google द्वारा विकसित एक समाचार एग्रीगेटर सेवा है। यह हजारों प्रकाशकों और पत्रिकाओं से व्यवस्थित लेखों के लिंक का निरंतर प्रवाह प्रस्तुत करता है। Google News एंड्रॉइड, iOS और वेब पर एक ऐप के रूप में उपलब्ध है। Google ने सितंबर 2002 में बीटा संस्करण और जनवरी 2006 में आधिकारिक ऐप जारी किया।"""
tamil_text = """கூகுள் செய்திகள் என்பது கூகுள் உருவாக்கிய செய்தி சேகரிப்பு சேவையாகும். ஆயிரக்கணக்கான வெளியீட்டாளர்கள் மற்றும் பத்திரிகைகளிலிருந்து ஒழுங்கமைக்கப்பட்ட கட்டுரைகளுக்கான தொடர்ச்சியான இணைப்புகளை இது வழங்குகிறது. Android, iOS மற்றும் இணையத்தில் Google செய்திகள் ஒரு பயன்பாடாகக் கிடைக்கிறது. கூகுள் செப்டம்பர் 2002 இல் பீட்டா பதிப்பையும், ஜனவரி 2006 இல் அதிகாரப்பூர்வ பயன்பாட்டையும் வெளியிட்டது."""
malayalam_text = """Google വികസിപ്പിച്ച വാർത്താ അഗ്രഗേറ്റർ സേവനമാണ് Google News. ആയിരക്കണക്കിന് പ്രസാധകരിൽ നിന്നും മാസികകളിൽ നിന്നും സംഘടിപ്പിച്ച ലേഖനങ്ങളിലേക്കുള്ള ലിങ്കുകളുടെ തുടർച്ചയായ ഒഴുക്ക് ഇത് അവതരിപ്പിക്കുന്നു. Android, iOS, Web എന്നിവയിൽ Google News ഒരു ആപ്പായി ലഭ്യമാണ്. ഗൂഗിൾ 2002 സെപ്റ്റംബറിൽ ഒരു ബീറ്റ പതിപ്പും 2006 ജനുവരിയിൽ ഔദ്യോഗിക ആപ്പും പുറത്തിറക്കി."""
gujarati_text = """ગૂગલ ન્યૂઝ એ ગૂગલ દ્વારા વિકસિત ન્યૂઝ એગ્રીગેટર સેવા છે. તે હજારો પ્રકાશકો અને સામયિકોમાંથી આયોજિત લેખોની લિંક્સનો સતત પ્રવાહ રજૂ કરે છે. Google News એ Android, iOS અને વેબ પર એપ્લિકેશન તરીકે ઉપલબ્ધ છે. ગૂગલે સપ્ટેમ્બર 2002માં બીટા વર્ઝન અને જાન્યુઆરી 2006માં સત્તાવાર એપ બહાર પાડી."""
bengali_text = """গুগল নিউজ হল একটি নিউজ এগ্রিগেটর সার্ভিস যা গুগল ডেভেলপ করেছে। এটি হাজার হাজার প্রকাশক এবং ম্যাগাজিন থেকে সংগঠিত নিবন্ধগুলির লিঙ্কগুলির একটি অবিচ্ছিন্ন প্রবাহ উপস্থাপন করে। Google News Android, iOS এবং ওয়েবে একটি অ্যাপ হিসেবে উপলব্ধ। গুগল 2002 সালের সেপ্টেম্বরে একটি বিটা সংস্করণ এবং জানুয়ারী 2006 সালে অফিসিয়াল অ্যাপ প্রকাশ করে।"""
odia_text = """ଗୁଲ୍ ଦ୍ୱାରା ବିକଶିତ ଏକ ନ୍ୟୁଜ୍ ଏଗ୍ରିଗେଟର୍ ସେବା | ଏହା ହଜାର ହଜାର ପ୍ରକାଶକ ଏବଂ ପତ୍ରିକାଗୁଡ଼ିକରୁ ସଂଗଠିତ ପ୍ରବନ୍ଧଗୁଡିକ ସହିତ ଲିଙ୍କଗୁଡିକର କ୍ରମାଗତ ପ୍ରବାହକୁ ଉପସ୍ଥାପନ କରେ | ଗୁଗୁଲ୍ ନ୍ୟୁଜ୍ ଆଣ୍ଡ୍ରଏଡ୍, ଆଇଓଏସ୍ ଏବଂ ୱେବରେ ଏକ ଆପ୍ ଭାବରେ ଉପଲବ୍ଧ | ସେପ୍ଟେମ୍ବର 2002 ରେ ଗୁଗୁଲ୍ ଏକ ବିଟା ସଂସ୍କରଣ ଏବଂ ଜାନୁଆରୀ 2006 ରେ ଅଫିସିଆଲ୍ ଆପ୍ ପ୍ରକାଶ କରିଥିଲା ​​|"""
telugu_text = """Google వార్తలు అనేది Google ద్వారా అభివృద్ధి చేయబడిన వార్తా అగ్రిగేటర్ సేవ. ఇది వేలకొద్దీ ప్రచురణకర్తలు మరియు మ్యాగజైన్‌ల నుండి నిర్వహించబడిన కథనాలకు నిరంతర లింక్‌లను అందిస్తుంది. Google వార్తలు Android, iOS మరియు వెబ్‌లో యాప్‌గా అందుబాటులో ఉన్నాయి. గూగుల్ సెప్టెంబరు 2002లో బీటా వెర్షన్‌ను మరియు జనవరి 2006లో అధికారిక యాప్‌ను విడుదల చేసింది."""
punjabi_text = """ਗੂਗਲ ਨਿਊਜ਼ ਗੂਗਲ ਦੁਆਰਾ ਵਿਕਸਿਤ ਕੀਤੀ ਗਈ ਇੱਕ ਨਿਊਜ਼ ਐਗਰੀਗੇਟਰ ਸੇਵਾ ਹੈ। ਇਹ ਹਜ਼ਾਰਾਂ ਪ੍ਰਕਾਸ਼ਕਾਂ ਅਤੇ ਰਸਾਲਿਆਂ ਤੋਂ ਸੰਗਠਿਤ ਲੇਖਾਂ ਦੇ ਲਿੰਕਾਂ ਦਾ ਨਿਰੰਤਰ ਪ੍ਰਵਾਹ ਪੇਸ਼ ਕਰਦਾ ਹੈ। Google News Android, iOS ਅਤੇ ਵੈੱਬ 'ਤੇ ਇੱਕ ਐਪ ਵਜੋਂ ਉਪਲਬਧ ਹੈ। ਗੂਗਲ ਨੇ ਸਤੰਬਰ 2002 ਵਿੱਚ ਇੱਕ ਬੀਟਾ ਸੰਸਕਰਣ ਅਤੇ ਜਨਵਰੀ 2006 ਵਿੱਚ ਅਧਿਕਾਰਤ ਐਪ ਜਾਰੀ ਕੀਤਾ।"""
kannada_text = """Google News Google ನಿಂದ ಅಭಿವೃದ್ಧಿಪಡಿಸಲಾದ ಸುದ್ದಿ ಸಂಗ್ರಾಹಕ ಸೇವೆಯಾಗಿದೆ. ಇದು ಸಾವಿರಾರು ಪ್ರಕಾಶಕರು ಮತ್ತು ನಿಯತಕಾಲಿಕೆಗಳಿಂದ ಆಯೋಜಿಸಲಾದ ಲೇಖನಗಳಿಗೆ ನಿರಂತರವಾದ ಲಿಂಕ್‌ಗಳನ್ನು ಪ್ರಸ್ತುತಪಡಿಸುತ್ತದೆ. Android, iOS ಮತ್ತು ವೆಬ್‌ನಲ್ಲಿ Google News ಅಪ್ಲಿಕೇಶನ್‌ನಂತೆ ಲಭ್ಯವಿದೆ. ಗೂಗಲ್ ಸೆಪ್ಟೆಂಬರ್ 2002 ರಲ್ಲಿ ಬೀಟಾ ಆವೃತ್ತಿಯನ್ನು ಮತ್ತು ಜನವರಿ 2006 ರಲ್ಲಿ ಅಧಿಕೃತ ಅಪ್ಲಿಕೇಶನ್ ಅನ್ನು ಬಿಡುಗಡೆ ಮಾಡಿತು."""

In [ ]:
language_text = {
         'hindi': hindi_text, 
         'tamil': tamil_text, 
         'malayalam': malayalam_text, 
         'gujarati': gujarati_text, 
         'bengali': bengali_text, 
         'odia': odia_text, 
         'telugu': telugu_text, 
         'punjabi': punjabi_text, 
         'kannada': kannada_text
        }

for language in language_text:
    print("Language: ", language)
    print(tokenizer.tokenize(language_text[language][:50]))
    print("------------------------------------------------------------")

In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset
import json
import pandas as pd

### Prompt

In [ ]:
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

### EOS_TOKEN

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

### Prompts 

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is None:
            input = ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

def formatting_prompts_func_malayalam(examples):
    data = examples["Prompt"]
    texts = []
    for text in data:
        if text is None:
            pass
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = text + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

def formatting_prompts_func_telugu(examples):
    instructions = examples["telugu_instruction"]
    inputs       = examples["telugu_input"]
    outputs      = examples["telugu_output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is None:
            input = ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
def process_dataset(dataset_name, 
                    split_type, 
                    processing_func, 
                    rename_column = False, 
                    filter_data = False, 
                    filter_column_value = 'id', 
                    filter_value = 'alpaca',
                    num_samples=20000):
    
    if isinstance(dataset_name, str):
        dataset = load_dataset(dataset_name, split=split_type)
    else:
        # Assuming dataset_name is a filepath for JSON file
        with open(dataset_name, 'r') as file:
            data = []
            for line_number, line in enumerate(file, 1):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON at line {line_number}: {e}")
            dataset = Dataset.from_pandas(pd.DataFrame(data))
    
    if post_processing_steps:
        for post_processing_step in post_processing_steps:
            dataset = post_processing_step(dataset)
    
    if rename_column:
        dataset = rename(dataset)
    
    if filter_data:
        dataset = filter_dataset(dataset, num_samples, value, column_name)
    
    dataset = dataset.map(processing_func, batched=True)
    
    return dataset

# Define the additional processing steps
def rename(dataset):
    return dataset.rename_column('response', 'output')

def filter_dataset(dataset, num_samples, value, column_name):
    return dataset.filter(lambda example: value in example[column_name]).shuffle(seed=42).select(range(num_samples))

### Process each language dataset

#### English

In [ ]:
dataset_en = process_dataset("yahma/alpaca-cleaned", "train", formatting_prompts_func)

print(dataset_en['text'][0])

#### Gujarati

In [ ]:
dataset_guj = process_dataset('gujarati.json', "train", formatting_prompts_func)
print(dataset_guj['text'][0])

#### Kannada

In [ ]:
dataset_kn1 = process_dataset("Tensoic/airoboros-3.2_kn", "train", formatting_prompts_func)

dataset_kn2 = process_dataset("Tensoic/gpt-teacher_kn", "train", formatting_prompts_func, True)

print(dataset_kn1['text'][0])

print(dataset_kn2['text'][0])

#### Malayalam

In [ ]:
dataset_mal = process_dataset("VishnuPJ/Alpaca_Instruct_Malayalam", "train", formatting_prompts_func_malayalam)

print(dataset_mal['text'][0])

#### Hindi

In [ ]:
dataset_hi1 = process_dataset("ravithejads/samvaad-hi-filtered", "train", formatting_prompts_func)

dataset_hi2 = process_dataset("HydraIndicLM/hindi_alpaca_dolly_67k", "train", formatting_prompts_func, filter_data = True)

print(dataset_hi1['text'][0])

print(dataset_hi2['text'][0])

#### Odia

In [ ]:
dataset_odi1 = process_dataset("OdiaGenAI/Odia_Alpaca_instructions_52k", "train", formatting_prompts_func)

dataset_odi2 = process_dataset("OdiaGenAI/gpt-teacher-roleplay-odia-3k", "train", formatting_prompts_func, rename = True)

print(dataset_odi1['text'][0])

print(dataset_odi2['text'][0])

#### Punjabi

In [ ]:
dataset_pun = process_dataset("HydraIndicLM/punjabi_alpaca_52K", "train", formatting_prompts_func)

print(dataset_pun['text'][0])

#### Bengali

In [ ]:
dataset_ben = process_dataset("HydraIndicLM/bengali_alpaca_dolly_67k", "train", formatting_prompts_func, filter_data = True)

print(dataset_ben['text'][0])

#### Tamil

In [ ]:
print("Language: ")

dataset_ta = process_dataset("abhinand/tamil-alpaca", "train", formatting_prompts_func)

print(dataset_ta['text'][0])

#### Telugu

In [ ]:
dataset_te1 = process_dataset("Telugu-LLM-Labs/yahma_alpaca_cleaned_telugu_filtered_and_romanized", "train", formatting_prompts_func_telugu)

dataset_te2 = process_dataset("Telugu-LLM-Labs/teknium_GPTeacher_general_instruct_telugu_filtered_and_romanized", "train", formatting_prompts_func_telugu)

print(dataset_te1['text'][0])

print(dataset_te2['text'][0])

### Final dataset concatenation

In [ ]:
dataset = concatenate_datasets([
    dataset_en, dataset_guj, dataset_kn1, dataset_kn2, dataset_mal, dataset_hi1, dataset_hi2,
    dataset_pun, dataset_ben, dataset_ta, dataset_te1, dataset_te2, dataset_odi1, dataset_odi2
])

### Shuffle dataset

In [ ]:
print(dataset['text'][100])

In [ ]:
dataset_sft = dataset.shuffle(seed=42)

In [ ]:
print(dataset_sft['text'][100])

### Count tokens

In [ ]:
tokens = tokenizer.tokenize(dataset_sft['text'][100])
len(tokens)

### Hyper parameters

In [ ]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 8

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 1.0

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 10

# Pack multiple short examples in the same input sequence to increase efficiency and make training 5x faster for short sequences.
packing = True

# text field in dataset
dataset_text_field = "text"

# dataset para,
dataset_num_proc = 2

# Load the entire model on the GPU 0
# device_map = {"": 0}
device_map = "auto"

# monitoring
report_to = "wandb"

### TrainingArguments

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to=report_to,
)

### SFTTrainer

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_sft,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    dataset_num_proc = dataset_num_proc,
    packing = packing,
    args = training_arguments,
)

### Current memory stats

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

### Monitoring

In [ ]:
import wandb
wandb.init(project="<project>", name="<name>")

### Start training

In [ ]:
trainer_stats = trainer.train()

### Save model

In [ ]:
model.save_pretrained("<FINE_TUNED_MODEL_NAME>") # Local saving
model.push_to_hub("<HF_REPO_NAME>/<FINE_TUNED_MODEL_NAME>", token = '<HF_WRITE_TOKEN>') 
tokenizer.push_to_hub("<HF_REPO_NAME>/<FINE_TUNED_MODEL_NAME>", token = '<HF_WRITE_TOKEN>') 

### Test Query

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "(9+1)+(5+0). इसे 3 चरणों में हल करें.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
print(tokenizer.batch_decode(outputs))